# Day 4: Workflows and Agents

<div style="text-align:center;">
    <img src="../assets/wf_a.png" width="" height=""/>
</div>

**Summary:** In this notebook, we will start learning about agents. How we can use Langhchain and a new tool called [Langgraph](https://langchain-ai.github.io/langgraph/concepts/why-langgraph/#learn-langgraph-basics) to build agentic systems. In addition, we will start learning about Workflows.

Recommended Reading:
+ [Building Effective Agent - Anthropic](https://www.anthropic.com/engineering/building-effective-agents)

# Part 1: Agents

One way to think about agents is an LLM that has access to a tool or tools. Whenever it needs that tool to answer a question it uses the tool. If it doesn't need the tool to answer a question it doesn't use it. 

For example, what if ChatGPT could use a calculator to answer an addition question or if it could Google a fact if it needed more information about something to answer a question.


<div style="text-align:center;">
    <img src="../assets/agents.png" width="" height=""/>
</div>


**Tools:** You can think of tools as functions. Your tools can be something as simple as addition or something more complicated like searching the web to find results. We'll be using Langchain to build out our tools, which will look like functuions and pass it to the LLM for use. 

Then based on our question the LLM will decide on whether it makes sense to use of the tool. Langchain make it easy with the `@tool` decorator. Let's start...

In [47]:
import os
from dotenv import load_dotenv
load_dotenv()

from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [46]:
openai_api_key = os.environ.get("OPENAI_API_KEY")
openai_organization = os.environ.get("OPENAI_ORGANIZATION")

In [16]:
# Step 1: Define an additional tool. 
#    Out first tool will take two numbers add them and return the result

@tool
def addition(x: int, y: int) -> int:
    """This function is used to add two numbers together and return the result"""
    return x + y

# Step 2: Add tools to your list of tools
tools = [addition]

# Step 3: Instantiate a model
llm = ChatOpenAI(
    openai_api_key = openai_api_key, 
    openai_organization = openai_organization,
    model = "gpt-4o-mini")

# Step 4: Create an agent
agent = create_react_agent(llm, tools=tools)

In [33]:
# Step 5: Use your agent + tool to ask a question about addition.
response = agent.invoke({"messages":"What is 1 + 1?"})

In [34]:
for msg in response["messages"]:
    print("-------------------")
    print(msg)

-------------------
content='What is 1 + 1?' additional_kwargs={} response_metadata={} id='bbda38aa-0306-44a0-8b32-a9e47f776943'
-------------------
content='' additional_kwargs={'tool_calls': [{'id': 'call_0uLGnWGzJvF7YfQmwgFvcPbl', 'function': {'arguments': '{"x":1,"y":1}', 'name': 'addition'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--b98ea272-ebb0-422b-89d1-3e2db6c135a4-0' tool_calls=[{'name': 'addition', 'args': {'x': 1, 'y': 1}, 'id': 'call_0uLGnWGzJvF7YfQmwgFvcPbl', 'type': 'tool_call'}] usage_metadata={'input_tokens': 62, 'output_tokens': 18, 't

In [35]:
# Let's test our agent on questions that don't have to do with addition....
response = agent.invoke({"messages":"What is the capital of Indonesia?"})

In [36]:
for msg in response["messages"]:
    print("-------------------")
    print(msg)

-------------------
content='What is the capital of Indonesia?' additional_kwargs={} response_metadata={} id='b3264ea2-0b6c-4faa-ba23-7c7bd5140e75'
-------------------
content='The capital of Indonesia is Jakarta.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 61, 'total_tokens': 69, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None} id='run--cda803ad-65ed-42cb-9d48-55d947f1e8b2-0' usage_metadata={'input_tokens': 61, 'output_tokens': 8, 'total_tokens': 69, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### Let's look at what just happend

Question 1: What is 1 + 1?

```mermaid
flowchart LR
    node1[Question 1]
    node2[LLM asks is tool needed?]
    node3[Agent]
    node4[Agent]
    node5[Answer]
    node6[addition tool]
    node7[Answer]
    node1 ---> node2
    node2 -- No --> node3
    node2 -- Yes --> node4
    node3 --> node5
    node4 <--> node6
    node4 ---> node7

   subgraph highlight [" "]
        node4
        node6
        node7
    end
```
---
Question 2: What is the capital of Indonesia?

```mermaid
flowchart LR
    node1[Question 2]
    node2[LLM asks is tool needed?]
    node3[Agent]
    node4[Agent]
    node5[Answer]
    node6[addition tool]
    node7[Answer]
    node1 ---> node2
    node2 -- No --> node3
    node2 -- Yes --> node4
    node3 --> node5
    node4 <--> node6
    node4 ---> node7

    subgraph highlight [" "]
        node3
        node5
    end
    
```

# Step 2: More About Tools...

+ We do not need to limit the amount of tools to use to just one. We can set up multiple different tools for different things so let's try that.

In [149]:
@tool
def multiplication(x: int, y: int) -> int:
    """Use the tool whenever you need to multiply two different numbers and return the result"""
    return x * y

@tool
def celsius_to_fahrenheit(c: float):
    " Converts celsius to fahrenheit "
    return (c * 9/5) + 32 

@tool
def vector_search(question: str) -> str:
    """ Function whenever a user is asking about querying my first database """
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key,
                                  model="text-embedding-3-large",
                                  chunk_size=1000)
    
    vector_store = Chroma(collection_name="tumo_2025",
                          embedding_function=embeddings,
                          persist_directory="../db/my_first_vdb/")
    
    retriever = vector_store.as_retriever(search_kwargs={"k": 10})
    
    return retriever.invoke(question)

In [150]:
tools = [multiplication, celsius_to_fahrenheit, vector_search]

In [153]:
llm = ChatOpenAI(
    openai_api_key = openai_api_key, 
    openai_organization = openai_organization,
    model = "gpt-4o-mini")

agent = create_react_agent(llm, tools=tools)

In [154]:
agent.invoke({"messages":"Convert 32 degrees Celsius to Fahrenheit"})

{'messages': [HumanMessage(content='Convert 32 degrees Celsius to Fahrenheit', additional_kwargs={}, response_metadata={}, id='2183cc42-07bb-40c2-99fb-280c742450d2'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RAyE2rE7260zAxD56EnTKK85', 'function': {'arguments': '{"c":32}', 'name': 'celsius_to_fahrenheit'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 113, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cbc0024e-c55e-4d07-9319-0279fb1d100e-0', tool_calls=[{'name': 'celsius_to_fahrenheit', 'args': {'c': 32}, 'id': 'call_RAyE2rE7260zAxD56EnTKK85', 'type': 'tool_call'}], usage_me

In [156]:
agent.invoke({"messages":"Summarize the article on Ferrari stored in my vector database. Make sure to summarize the aticle in 3-4 sentences"})

{'messages': [HumanMessage(content='Summarize the article on Ferrari stored in my vector database. Make sure to summarize the aticle in 3-4 sentences', additional_kwargs={}, response_metadata={}, id='5c69f03b-467f-4d42-b17e-09f7e42c4b17'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LBLb8PC95sb5b6bkZ0xt3OZL', 'function': {'arguments': '{"question":"Ferrari article summary"}', 'name': 'vector_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 132, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--50a0e828-c831-42bc-bb2e-395a6d688563-0', tool_calls=[{'name': 'vector_search', 'a